<a href="https://colab.research.google.com/github/kwhuang8123/ck_scrap/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/mtginvestment/ck

/content/drive/My Drive/mtginvestment/ck


In [3]:
%ls

0504Goldfish.ipynb  ck_test/  raw_list.csv       raw_list.xlsx
0520Goldfish.ipynb  data/     raw_list_t.csv     service/
apt.log             pip.log   raw_list_t.gsheet  test.csv


In [4]:
#chromnium更新
import os
import re
import subprocess
import requests

# The deb files we need to install
deb_files_startstwith = [
    "chromium-codecs-ffmpeg-extra_",
    "chromium-codecs-ffmpeg_",
    "chromium-browser_",
    "chromium-chromedriver_"
]

def get_latest_version() -> str:
    # A request to security.ubuntu.com for getting latest version of chromium-browser
    # e.g. "112.0.5615.49-0ubuntu0.18.04.1_amd64.deb"
    url = "http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="chromium\-browser_([^"]+.ubuntu0\.18\.04\.1_amd64\.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise Exception("Can not find latest version!")
    return latest_version

def download(latest_version: str, quiet: bool):
    deb_files = []
    for deb_file in deb_files_startstwith:
        deb_files.append(deb_file + latest_version)

    for deb_file in deb_files:
        url = f"http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/{deb_file}"

        # Download deb file
        if quiet:
            command = f"wget -q -O /content/{deb_file} {url}"
        else:
            command = f"wget -O /content/{deb_file} {url}"
        print(f"Downloading: {deb_file}")
        # os.system(command)
        !$command

        # Install deb file
        if quiet:
            command = f"apt-get install /content/{deb_file} >> apt.log"
        else:
            command = f"apt-get install /content/{deb_file}"
        print(f"Installing: {deb_file}\n")
        # os.system(command)
        !$command

        # Delete deb file from disk
        os.remove(f"/content/{deb_file}")

def check_chromium_installation():
    try:
        subprocess.call(["chromium-browser"])
        print("Chromium installation successfull.")
    except FileNotFoundError:
        print("Chromium Installation Failed!")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium-browser for ubuntu 18.04
    latest_version = get_latest_version()
    # Download and install chromium-browser for ubuntu 20.04
    download(latest_version, quiet)
    # Check if installation succesfull
    check_chromium_installation()
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Chromium installation successfull.


In [5]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install fake_useragent

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,252 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,061 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,219 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [28.1 kB]
Get:13 https://ppa.launc

In [6]:
from service.ck2gf import ck2gf
from service.goldfish  import goldfish
from bs4 import BeautifulSoup
from matplotlib.pyplot import table
from csv import reader
from multiprocessing import Pool
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from operator import itemgetter, attrgetter
import requests
import bs4
import re
import ast
import time
import random
import pandas as pd
import threading
import queue
import csv
import os

In [7]:
class Worker(threading.Thread):
  def __init__(self, queue):
    threading.Thread.__init__(self)
    self.queue = queue

  def run(self):
    while self.queue.qsize() > 0:
      # 取得新的資料

        arg = self.queue.get(timeout=15)


        ck.get_price(cardkingdom, arg)




In [12]:
class ck():
  def __init__(self):
      super().__init__()
      self.original = []
      self.raw_list = []  #[name, 未處理setname, ck_price]
      self.main_list = [] #[name, 處理好的setname, ck_price]
      self.result_list = []#[name, setname, ck_price, gf_price, rate]
      self.tmp_list = []
      self.None_list = []
      self.false_list = []
      self.wrong_page=[]
      self.status=False
      self.false_404=[]
      self.start_time = time.time()
      self.end_time = time.time()
      self.error=0
      self.false_url=[]
      self.watch_list=[]
      self.count=0
      self.Create_time=time.strftime("%Y%m%d", time.localtime()) #現在時間
      self.thread=4

  def start(self):#目前先用手動註解掉


      #爬取主要表格
      self.get_data()
      df = pd.DataFrame(self.raw_list)
      df.to_csv('/content/drive/MyDrive/mtginvestment/ck/raw_list_t.csv',index=False,header=False )

      #self.readcsv()       # 讀取已經存好的csv檔(全部執行的話就要註解調不然會有同時出現兩個)
      self.filter_data()     # 過濾掉價格小於2的資料
      self.search_thread()    # goldfish查詢
      self.save()
      self.end_time = time.time()

      print(len(self.raw_list),'筆Raw_list')
      print(self.num+'筆資料查詢成功')
      print(len(self.None_list),'筆原本轉換為None')
      print(len(self.false_list),'筆錯誤')
      print(len(self.watch_list),'筆觀察清單')
      print('共',self.count,'筆資料')
      print('運行時間:'+str('%.2f'%(self.end_time-self.start_time))+'秒')

  def get_data(self):
    self.cul=0

    def click(Thread,start,n):
        # url = 'https://www.cardkingdom.com/purchasing/mtg_singles?filter%5Bsort%5D=price_desc&page='
        url = 'https://www.cardkingdom.com/purchasing/mtg_singles?filter%5Bsort%5D=price_desc&filter%5Bsearch%5D=mtg_advanced&filter%5Bname%5D=&filter%5Bedition%5D=&filter%5Bformat%5D=&filter%5Bsingles%5D=1&filter%5Bprice_op%5D=gte&filter%5Bprice%5D=1.99&page='
        # options = webdriver.ChromeOptions()
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)
        #先載入網站，一開始直接指定頁數會跳掉
        driver.get(url)
        for i in range(start,int(start+n)):
            new_url=url+str(i)
            try:
                driver.get(new_url)
                #此處有些微更改，新版selenium需使用By.來查詢類別
                iframe=driver.find_elements(By.CLASS_NAME,"itemContentWrapper")
                for j in iframe:
                    self.original.append(j.text)
                time.sleep(random.randint(1,4))

            except:
                self.wrong_page.append(i)
                print(f'{Thread} Page {i} fail')
                continue

            print(f'{Thread} : {str(i)} / {start}-{start+n-1} {100*(i-start+1)/n}%  {self.cul+len(self.original)}')
        driver.close()
        print(f'{Thread} : {start}-{start+n-1} Done')
    #由於Image 需要時間載入 故用Multi-thred多開分頁方式加快運行
    # 建立 n 個子執行緒
    n_thread=self.thread
    threads = []
    for i,j in zip( range(0,n_thread) , range(1,40,int(40/n_thread)) ):#測試中 原來用400
        threads.append(threading.Thread( target=click, args=("Thread-"+str(i) , j , 40/n_thread ) ) )
        threads[i].start()

        time.sleep(6)

    for i in range(n_thread):
      threads[i].join()
    #過濾資料
    self.re_data()

  def re_data(self):#正則 過濾資料
      for i in self.original:
          sepdata = str(re.findall('(.*)\n', i, re.MULTILINE))# e.g: 'Underground Sea\n3rd Edition (R)\nLand - Island Swamp\n$570.00 $741.00\nQty to Sell'
          sepdata = ast.literal_eval(sepdata)#str轉回list
          name = sepdata[0]
          edition = sepdata[1][:-4]
          price = str(re.findall('\$(.*)\$', str(sepdata[-2]), re.MULTILINE))
          price = price[2:-2]
          print(name, edition, price, sep='\t')
          tmp = [name, edition, price]
          self.raw_list.append(tmp)
      print("")
      # print(self.raw_list)

  def readcsv(self):#
      import csv
      with open('/content/drive/MyDrive/mtginvestment/ck/raw_list_t.csv', 'r') as csvfile:#raw_list_t = 測試用的小資料
        csv_reader = reader(csvfile)
        tmp = csv_reader
        for i in tmp:
          self.raw_list.append(i)
        csvfile.close()

  def filter_data(self): #過濾price小於2的資料
      for i in range(len(self.raw_list)-1,0,-1):
        if bool(self.raw_list[i][2].count(',')) == True:
          del self.raw_list[i]
        if float(self.raw_list[i][2])<=2:
          del self.raw_list[i]
      print(len(self.raw_list) , '筆資料待查詢')

  def search_thread(self):
      url_list = []
      #修正版本名稱至可用goldfish查詢

      for i in self.raw_list:
        try:
          setname = ck2gf(i[1]) #如果有跳key error通常是連接詞的大小寫有錯 不想理他的話就直接建一個他的dict給404就好了
          setname = setname.replace(':', '')
          setname = setname.replace("'", '')
          setname = setname.replace(' ', '+')
          tmp = [i[0], setname, i[2]]
          self.main_list.append(tmp)
        except KeyError as error:
          pass

      #goldfish查詢
      for i in self.main_list:
          url = goldfish.get_url(i[1], i[0], '')
          tmp = [i[0], i[1], i[2], url]
          url_list.append(tmp)

      self.my_queue = queue.Queue()

      for i in url_list:
          self.my_queue.put(i)

      self.work()

      self.num = str(len(self.result_list))



  def get_price(self, url):
      #fake-useragent 套件可以隨機產生 User-Agent 字串
      ua = UserAgent()
      user_agent = ua.random
      headers = {'user-agent': user_agent}
      #false_list: 錯誤 常見部分:404, 200(但沒中價)
      #none_list: 自行過濾太複雜的版本

      if url[3] :#從url過濾是否能查到價格

          self.count+=1
          res = requests.get(url[3], headers=headers)
          soup = BeautifulSoup(res.text, 'html.parser')
          b = soup.find_all('div', class_ = 'price-box-price')#中價

          if b:# 有些它的版本沒有分那麼細 如果沒查到東西就先忽略
              mid = b[0].text
              if 'tix' not in mid:#沒閃的版本會查到mo價格
                      mid = mid.replace(',', '')
                      mid = mid[2:]
                      ck_price = float(url[2])
                      rate = ck_price / float(mid) * 30
                      tmp = [url[0], url[1], url[2], mid, '%.2f'%rate]
                      self.result_list.append(tmp)
                      print(self.count,tmp)
                      if rate>=20:
                        self.watch_list.append(tmp)
          else:
            print(self.count,"False", res.status_code, b)
            self.error+=1
            self.false_list.append([url[0], url[1], url[2], url[3], res.status_code])

      if url[3]==None:
        self.count+=1
        self.None_list.append(url)
        print(self.count, "None")

      time.sleep(4)
  def work(self):

      # 建立 Worker
      my_worker1 = Worker(self.my_queue)
      my_worker2 = Worker(self.my_queue)
      my_worker3 = Worker(self.my_queue)
      my_worker4 = Worker(self.my_queue)
      my_worker5 = Worker(self.my_queue)
      my_worker6 = Worker(self.my_queue)

      # 讓 Worker 開始處理資料
      my_worker1.start()
      my_worker2.start()
      my_worker3.start()
      my_worker4.start()
      my_worker5.start()
      my_worker6.start()
      # 等待所有 Worker 結束
      my_worker1.join()
      my_worker2.join()
      my_worker3.join()
      my_worker4.join()
      my_worker5.join()
      my_worker6.join()

  def save(self):
      path = '/content/drive/MyDrive/mtginvestment/ck/data/'+self.Create_time
      if not os.path.isdir(path):
        os.mkdir(path)

      self.watch_list=sorted(self.watch_list, key = itemgetter(4),reverse=True)

      with open(path + '/result.csv', 'w',  newline='', encoding= 'utf-8') as output:
              csv_writer = csv.writer(output)
              for i in self.result_list:
                  csv_writer.writerow(i)
      with open(path + '/false_list.csv', 'w',  newline='', encoding= 'utf-8') as output_false_list:
              csv_writer = csv.writer(output_false_list)
              for i in self.false_list:
                  csv_writer.writerow(i)
      with open(path + '/None_list.csv', 'w',  newline='', encoding= 'utf-8') as output_None_list:
              csv_writer = csv.writer(output_None_list)
              for i in self.None_list:
                  csv_writer.writerow(i)
      # with open(path + '/false_404.csv', 'w',  newline='', encoding= 'utf-8') as output_false_404:
      #         csv_writer = csv.writer(output_false_404)
      #         for i in self.false_404:
      #             csv_writer.writerow(i)

      with open(path + '/watch_list.csv', 'w',  newline='', encoding= 'utf-8') as output_watch_list:
              csv_writer = csv.writer(output_watch_list)
              for i in self.watch_list:
                  csv_writer.writerow(i)

if __name__ =='__main__':
    cardkingdom = ck()
    cardkingdom.start()

Thread-0 : 1 / 1-10.0 10.0%  25
Thread-1 : 11 / 11-20.0 10.0%  50
Thread-0 : 2 / 1-10.0 20.0%  78
Thread-2 : 21 / 21-30.0 10.0%  125
Thread-1 : 12 / 11-20.0 20.0%  126
Thread-0 : 3 / 1-10.0 30.0%  151
Thread-3 : 31 / 31-40.0 10.0%  223
Thread-2 : 22 / 21-30.0 20.0%  241
Thread-1 : 13 / 11-20.0 30.0%  243
Thread-0 : 4 / 1-10.0 40.0%  251
Thread-3 : 32 / 31-40.0 20.0%  286
Thread-2 : 23 / 21-30.0 30.0%  331
Thread-1 : 14 / 11-20.0 40.0%  347
Thread-0 : 5 / 1-10.0 50.0%  350
Thread-2 : 24 / 21-30.0 40.0%  413
Thread-3 : 33 / 31-40.0 30.0%  433
Thread-0 : 6 / 1-10.0 60.0%  453
Thread-1 : 15 / 11-20.0 50.0%  469
Thread-2 : 25 / 21-30.0 50.0%  500
Thread-3 : 34 / 31-40.0 40.0%  508
Thread-0 : 7 / 1-10.0 70.0%  541
Thread-1 : 16 / 11-20.0 60.0%  556
Thread-2 : 26 / 21-30.0 60.0%  600
Thread-3 : 35 / 31-40.0 50.0%  609
Thread-0 : 8 / 1-10.0 80.0%  642
Thread-1 : 17 / 11-20.0 70.0%  658
Thread-2 : 27 / 21-30.0 70.0%  694
Thread-3 : 36 / 31-40.0 60.0%  705
Thread-0 : 9 / 1-10.0 90.0%  751
Thread

In [ ]:
#original: 剛撈下來的資料
#raw_list: re的資料
#main_list: 經過re的資料
#false_url: 你的URL錯誤
#false_404: request成功 但是沒有撈到資料
#none_list: 你本來就寫404的
#watch_list: 你要的
#result: 成功的大表